In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from peft import PeftModel
import pandas as pd
import json

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = logits.argmax(dim=-1).item()
    
    return predicted_class_id

model_path = "../model/classifier_model"

tokenizer = BertTokenizer.from_pretrained(model_path)

base_model_name = "klue/bert-base"
base_model = BertForSequenceClassification.from_pretrained(base_model_name, num_labels=5)

model = PeftModel.from_pretrained(base_model, model_path)

model.eval()  

dic = []

test_cls = pd.read_json("../data/test_cls.json")

for question in test_cls['question']:
    pred_label = predict(question)
    dic.append({"question": question, "label": pred_label})

with open("../outputs/cls_output.json", 'w', encoding="utf-8") as f:
    json.dump(dic, f, ensure_ascii=False, indent=4)

c:\dev_repos\NLP_TermProject\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
